In [1]:
import numpy as np
import plotly.graph_objects as go

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import rsi_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=300,
)

datetimes = candles[:,CandleBodyType.Timestamp].astype('datetime64[ms]')
close_prices = candles[:,CandleBodyType.Close]

In [3]:
rsi_is_below = 50
rsi_length = 14

In [6]:
rsi = rsi_tv(
    source=close_prices,
    length=rsi_length
)

rsi = np.around(rsi,1)
rsi

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 54.90, 58.30, 59.00, 56.20, 56.50, 56.90, 49.00, 48.90, 51.70,
       47.30, 47.60, 54.10, 51.30, 48.80, 52.30, 48.80, 49.60, 51.50,
       57.80, 54.90, 43.50, 40.40, 36.80, 41.70, 49.10, 46.90, 50.40,
       54.10, 54.90, 65.60, 55.20, 52.50, 55.90, 58.00, 52.40, 53.80,
       53.10, 51.90, 50.90, 43.80, 40.90, 39.40, 39.00, 45.10, 47.20,
       43.30, 47.00, 42.80, 49.40, 46.20, 47.50, 42.90, 37.10, 42.50,
       37.50, 31.50, 28.90, 41.80, 40.80, 41.30, 38.30, 40.20, 45.50,
       52.50, 45.80, 50.70, 60.20, 72.40, 77.40, 80.80, 81.90, 76.90,
       79.20, 79.60, 79.80, 84.20, 85.70, 82.20, 82.70, 83.50, 83.90,
       80.90, 81.70, 78.10, 75.00, 76.40, 74.60, 77.00, 85.00, 87.70,
       85.60, 86.50, 67.90, 70.20, 65.80, 66.40, 68.80, 70.40, 71.00,
       63.90, 64.40, 67.70, 65.30, 67.40, 69.20, 69.60, 66.40, 66.60,
       66.20, 71.20, 67.70, 67.00, 71.20, 64.40, 66.00, 69.40, 57.70,
       59.50,

In [7]:
prev_rsi = np.roll(rsi,1)
prev_rsi[0] = np.nan

prev_prev_rsi = np.roll(prev_rsi,1)
prev_prev_rsi[0] = np.nan

In [8]:
falling = prev_prev_rsi > prev_rsi
rising = rsi > prev_rsi
is_below = rsi < rsi_is_below

In [9]:
entries = np.where(is_below & falling & rising, True, False)

In [10]:
entry_signals = np.where(entries, rsi, np.nan)

In [11]:
exit_prices = np.full_like(rsi, np.nan)

In [12]:
fig = go.Figure()

In [13]:
fig.add_scatter(
    x=datetimes,
    y=rsi,
    name="rsi",
    line_color="yellow",
)

In [14]:
fig.add_scatter(
    x=datetimes,
    y=entry_signals,
    mode="markers",
    name="entries",
    marker=dict(
        size=12,
        symbol="circle",
        color="#00F6FF",
        line=dict(
            width=1,
            color="DarkSlateGrey",
        ),
    ),
)

In [15]:
fig.add_hline(
    y=rsi_is_below,
    opacity=0.3,
    line_color="red",
)


In [16]:
fig.update_layout(
    height=500,
    xaxis_rangeslider_visible=False,
    title=dict(
        x=0.5,
        text="Long Signals",
        xanchor="center",
        font=dict(
            size=50,
        ),
    ),
)